In [0]:
import pandas as pd
import numpy as np
import io
import sys

In [0]:
from google.colab import  files
file = files.upload()

Saving 20190928-items.csv to 20190928-items.csv
Saving 20190928-reviews.csv to 20190928-reviews.csv


In [0]:
reviews = pd.read_csv(io.BytesIO(file['20190928-reviews.csv']))
items = pd.read_csv(io.BytesIO(file['20190928-items.csv']))
reviews

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0
...,...,...,...,...,...,...,...,...
82810,B07WXKWDT2,Prabhakaran,5,"September 24, 2019",True,International version. Will require USA adapte...,Best phone at this price.,NaN
82811,B07WXKWDT2,rvwonder,3,"September 17, 2019",False,Be careful of International Version if Using T...,If you intend to use this phone on T Mobile be...,NaN
82812,B07WXKWDT2,markw707,1,"September 8, 2019",False,Moto G7 Play Freezes Every Time I Use It.,Here is my Moto G7 Play complaint: It freezes ...,NaN
82813,B07WXKWDT2,Amazon Customer,1,"September 13, 2019",False,Camera is not good,As far as function works great camera no go wo...,NaN


In [0]:
reviews.dtypes

asin             object
name             object
rating            int64
date             object
verified           bool
title            object
body             object
helpfulVotes    float64
dtype: object

In [0]:
reviews.isnull().sum()

asin                0
name                5
rating              0
date                0
verified            0
title              10
body               20
helpfulVotes    49681
dtype: int64

In [0]:
reviews = reviews[reviews['title'].notnull()]
reviews = reviews[reviews['body'].notnull()]

In [0]:
reviews

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0
...,...,...,...,...,...,...,...,...
82810,B07WXKWDT2,Prabhakaran,5,"September 24, 2019",True,International version. Will require USA adapte...,Best phone at this price.,NaN
82811,B07WXKWDT2,rvwonder,3,"September 17, 2019",False,Be careful of International Version if Using T...,If you intend to use this phone on T Mobile be...,NaN
82812,B07WXKWDT2,markw707,1,"September 8, 2019",False,Moto G7 Play Freezes Every Time I Use It.,Here is my Moto G7 Play complaint: It freezes ...,NaN
82813,B07WXKWDT2,Amazon Customer,1,"September 13, 2019",False,Camera is not good,As far as function works great camera no go wo...,NaN


Converting the boolean values to integers.

In [0]:
reviews['helpfulVotes'] = reviews['helpfulVotes'].fillna(0)
def verified(x):
  if x==False:
    n=0
  else:
    n=1
  return n


In [0]:
reviews['verified'] = reviews['verified'].map(lambda x: verified(x))

In [0]:
reviews

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",0,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",0,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",0,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",0,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",0,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0
...,...,...,...,...,...,...,...,...
82810,B07WXKWDT2,Prabhakaran,5,"September 24, 2019",1,International version. Will require USA adapte...,Best phone at this price.,0.0
82811,B07WXKWDT2,rvwonder,3,"September 17, 2019",0,Be careful of International Version if Using T...,If you intend to use this phone on T Mobile be...,0.0
82812,B07WXKWDT2,markw707,1,"September 8, 2019",0,Moto G7 Play Freezes Every Time I Use It.,Here is my Moto G7 Play complaint: It freezes ...,0.0
82813,B07WXKWDT2,Amazon Customer,1,"September 13, 2019",0,Camera is not good,As far as function works great camera no go wo...,0.0


In [0]:
reviews.helpfulVotes = reviews.helpfulVotes.astype(int)

In [0]:
reviews

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",0,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",0,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17
2,B0000SX2UC,Brooke,5,"December 30, 2003",0,Love This Phone,"This is a great, reliable phone. I also purcha...",5
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",0,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",0,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1
...,...,...,...,...,...,...,...,...
82810,B07WXKWDT2,Prabhakaran,5,"September 24, 2019",1,International version. Will require USA adapte...,Best phone at this price.,0
82811,B07WXKWDT2,rvwonder,3,"September 17, 2019",0,Be careful of International Version if Using T...,If you intend to use this phone on T Mobile be...,0
82812,B07WXKWDT2,markw707,1,"September 8, 2019",0,Moto G7 Play Freezes Every Time I Use It.,Here is my Moto G7 Play complaint: It freezes ...,0
82813,B07WXKWDT2,Amazon Customer,1,"September 13, 2019",0,Camera is not good,As far as function works great camera no go wo...,0


Vectorizing the data.

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
a = tf.fit_transform(reviews['title'])

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test = train_test_split(a,train_size=0.8,random_state=1)
y_train,y_test = train_test_split(reviews['rating'],train_size=0.8,random_state=1)

Training and testing the frist subset of data using Random Forest CLassifier, MultinomialNB and K-Neighbors

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
from sklearn.metrics import accuracy_score as ac
g1 = ac(rf.predict(x_test),y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7204807634233255

In [0]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_train,y_train)
g2 = ac(nb.predict(x_test),y_test)

0.7091260494050855

In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)
g3 = ac(knn.predict(x_test),y_test)

0.6762094582351875

In [0]:
print('The accuracy scores of the algorithms are:\n1. Random Forest:',g1,'\n2. MultinomialNB:',g2,'\n3.K-Nearest Neighbors:',g3)

In [0]:
#from sklearn.svm import SVC
#svm = SVC(kernel='linear')
#svm.fit(x_train,y_train)
#ac(svm.predict(x_test),y_test)


Training and testing the second subset of data.

In [0]:
b = tf.fit_transform(reviews['body'])
x_train,x_test = train_test_split(b,train_size=0.8,random_state=1)
y_train,y_test = train_test_split(reviews['rating'],train_size=0.8,random_state=1)
rf.fit(x_train,y_train)
g4 = ac(rf.predict(x_test),y_test)
nb.fit(x_train,y_train)
g5 = ac(nb.predict(x_test),y_test)

0.6588754001328743

In [0]:
print(ac(rf.predict(x_test),y_test),ac(nb.predict(x_test),y_test))

0.676632240140122 0.6588754001328743


In [0]:
knn.fit(x_train,y_train)
g6 = ac(knn.predict(x_test),y_test)

0.5532403213142477

In [0]:
print('The accuracy scores of the algorithms are:\n1. Random Forest:',g4,'\n2. MultinomialNB:',g5,'\n3.K-Nearest Neighbors:',g6)

Training and testing third subset of data.

In [0]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=1)
a1 = nmf.fit_transform(a)
b1 = nmf.fit_transform(b)

In [0]:
df = pd.DataFrame({'title':a1[:,0],'body':b1[:,0],'verified':reviews['verified'],'votes':reviews['helpfulVotes']})
x_train,x_test = train_test_split(df,train_size=0.8,random_state=1)
y_train,y_test = train_test_split(reviews['rating'],train_size=0.8,random_state=1)
rf.fit(x_train,y_train)
nb.fit(x_train,y_train)
g7 = ac(rf.predict(x_test),y_test)
g8 = ac(nb.predict(x_test),y_test))

0.5812043244549133 0.5359062632119346


In [0]:
knn.fit(x_train,y_train)
g9 = ac(knn.predict(x_test),y_test)

0.5008757625173642

In [0]:
print('The accuracy scores of the algorithms are:\n1. Random Forest:',g7,'\n2. MultinomialNB:',g8,'\n3.K-Nearest Neighbors:',g9)

Training and testing fourth subset of data.

In [0]:
nmf1 = NMF(n_components=10)
a2 = nmf1.fit_transform(a)
b2 = nmf.fit_transform(b)

In [0]:
df1 = pd.DataFrame(a2)
df2 = pd.DataFrame(b2)
for i in range(df2.shape[1]):
  df1[i+df1.shape[1]] = df2[i]
df1[df1.shape[1]+1] = reviews['verified']
df1[df1.shape[1]+1] = reviews['verified']
df1
df1 = df1.fillna(0)

In [0]:
x_train,x_test = train_test_split(df1,train_size=0.8,random_state=1)
y_train,y_test = train_test_split(reviews['rating'],train_size=0.8,random_state=1)
rf.fit(x_train,y_train)
nb.fit(x_train,y_train)
knn.fit(x_train,y_train)
g10 = ac(rf.predict(x_test),y_test)
g11 = ac(nb.predict(x_test),y_test)
g12 = ac(knn.predict(x_test),y_test))

0.6680558072114513 0.5414628253910733 0.6086247508606631


In [0]:
print('The accuracy scores of the algorithms are:\n1. Random Forest:',g10,'\n2. MultinomialNB:',g11,'\n3.K-Nearest Neighbors:',g12)

**Topic Modeling**

In [0]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, PCA
from sklearn.preprocessing import normalize
import pickle

In [0]:
def top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

**LSA**

In [0]:
tf_idf = TfidfVectorizer(stop_words='english', max_features=1000)
data_tf_idf = tf_idf.fit_transform(reviews['body'])
nmf = NMF(n_components=30, random_state=1,
          alpha=.1, l1_ratio=.5)
nmf.fit(data_tf_idf)
nmf_data=nmf.transform(data_tf_idf)

In [0]:
top_words(nmf, tf_idf.get_feature_names(), 10)

Topic #0: camera samsung phones really iphone better time android ve quality
Topic #1: good quality really value deal pretty camera overall looks cellphone
Topic #2: great deal value camera phone quality looks shape features worked
Topic #3: love absolutely new thank note features size big phone galaxy
Topic #4: like new brand looks looked refurbished scratches really box look
Topic #5: excellent quality camera condition value service thanks recommend smartphone performance
Topic #6: phone cell received recommend amazing problems money got refurbished purchased
Topic #7: works perfectly fine looks issues problems time scratches mobile arrived
Topic #8: product described seller service recommend satisfied received time arrived thanks
Topic #9: nice really camera features phone big thanks smartphone size looking
Topic #10: perfect size thanks thank condition gift absolutely arrived exactly looks
Topic #11: el la muy que en es lo teléfono se buen
Topic #12: battery life charge long day ho

In [0]:
np.argmax(nmf_data[0])

0

**PLSA**

In [0]:
tf_idf = TfidfVectorizer(stop_words='english', max_features=1000)
data_tf_idf = tf_idf.fit_transform(reviews['body'])
nmf = NMF(n_components=30, random_state=1, beta_loss='kullback-leibler',
          solver='mu', alpha=.1, l1_ratio=.5)
nmf_data=nmf.fit(data_tf_idf)

In [0]:
top_words(nmf, tf_idf.get_feature_names(), 10)

Topic #0: phone like great don getting going thing lot make really
Topic #1: good pretty overall value choice really performance complain stuff real
Topic #2: great works deal value shape thanks freezes freezing friend friendly
Topic #3: love great absolutely color thank learning zero function friendly friends
Topic #4: new like works brand looks perfectly refurbished looked scratches fine
Topic #5: product excellent seller recommend service 100 highly satisfied customer recommended
Topic #6: phone cell perfectly cricket stolen shape metro pcs activated tmobile
Topic #7: battery camera screen day apps life android 10 using works
Topic #8: received return phone disappointed seller said service amazon tried sent
Topic #9: nice big really little looking clean wonderful cheap design durable
Topic #10: perfect condition thank arrived size absolutely needs near early fits
Topic #11: el muy la en lo es que buen bueno se
Topic #12: battery phone life charge day long hours days charged lasts
To

**LDA**

In [0]:
count = CountVectorizer(stop_words='english', max_features=1000)
data_count = count.fit_transform(reviews['body'])
lda = LatentDirichletAllocation(n_components=25, max_iter=5, 
                                learning_method='online', random_state=1)
lda_data=lda.fit_transform(data_count)

In [0]:
top_words(lda, count.get_feature_names(), 10)

Topic #0: new phone came like charger brand box refurbished looks scratches
Topic #1: great works phone excellent product recommend easy use highly s10
Topic #2: love phone price great awesome money wasn s5 waste delivery
Topic #3: phone issues warranty weeks couldn worked tried talk time straight
Topic #4: camera fast phone amazing really price best better quality super
Topic #5: moto nice el la muy que en lo se described
Topic #6: sim card unlocked phone mobile sprint locked work sd unlock
Topic #7: screen pixel use fingerprint photos usb case protector mode display
Topic #8: phone ve phones buy don nokia just time want sony
Topic #9: far happy phone problems bought started s7 old cell s9
Topic #10: perfect condition purchase phone received works quick completely amazon price
Topic #11: android apps google device app version apple update performance software
Topic #12: phone verizon service network customer store took absolutely data motorola
Topic #13: expected ok quality item littl

In [0]:
np.argmax(lda_data[5])

7

Training and testing the first subset of data using Neural Networks.

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding, LSTM, GRU, Input
from keras.layers import Conv2D, MaxPooling2D, Conv1D,MaxPooling1D
from keras import backend as K
from keras.preprocessing.text import Tokenizer, hashing_trick
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test = train_test_split(reviews['body'],train_size=0.8,random_state=1)
y_train,y_test = train_test_split(reviews['rating'],train_size=0.8,random_state=1)

In [0]:
x_train

53566                                                 good
32975    it broke the display that's under the exterior...
46861    This is an excellent phone which far exceeded ...
45368                    Not happy but at least it’s cheap
23459    This phone appeared to arrive NEW, factory shr...
                               ...                        
21444    Don't buy this iPhone plus. Touch screen is no...
73374    Well I gotta say I'm really surprised with wha...
50074    This phone shipped with Oreo 8.1. The mic woul...
5192           Came as described! Saved $70 not buying new
77734    Awesome for everyday use. Might be a little sm...
Name: body, Length: 66228, dtype: object

Cleaning the text in the 'body' column in the 'reviews' dataframe for tokenizing.

In [0]:
import re
def replace(x):
  n = re.sub(r'\d+','',x)
  return n


In [0]:
x_train.map(lambda x: replace(x))
x_test.map(lambda x: replace(x))

79022    RAM needs to be bigger. Internal Storage could...
47036    Duel SIM phone. Works well on T-Mobile and I u...
20568                                   slow as.... f.....
32461    Horrible! I waited for almost a week to receiv...
8292      Best phone available, until the Note  comes out.
                               ...                        
17758    Product was not as described. Returned it. It ...
77193                                          Reliability
64088    This is a nice budget phone !!!! So far i have...
2937                                             Very best
5315     El telefono se ve muy Bonito lo malo y lo peor...
Name: body, Length: 16557, dtype: object

In [0]:
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(x_train)
train_X = pad_sequences(tokenizer.texts_to_sequences(x_train), 33)
test_X = pad_sequences(tokenizer.texts_to_sequences(x_test),33)
train_Y=keras.utils.to_categorical(y_train)
test_Y=keras.utils.to_categorical(y_test)

In [0]:
embed_dim = 128
lstm_out = 196
max_features = 2000
model = Sequential()
model.add(Embedding(tokenizer.num_words, tokenizer.num_words, input_length=33))
model.add(Conv1D(66, kernel_size=3, activation='relu'))
model.add(Conv1D(66, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(.5))
model.add(Conv1D(33, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(.5))
model.add(Dense(6,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [0]:
batch_size = 32
model.fit( train_X,#.reshape((train_X.shape[0], train_X.shape[1],1)), 
           train_Y,
           epochs = 100, 
           batch_size=batch_size, 
           verbose = 1, 
           validation_data=(test_X, test_Y))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 66228 samples, validate on 16557 samples
Epoch 1/100





66228/66228 [==============================] - 42s 629us/step - loss: 1.0176 - acc: 0.6437 - val_loss: 0.9288 - val_acc: 0.6719
Epoch 2/100
66228/66228 [==============================] - 35s 524us/step - loss: 0.9154 - acc: 0.6763 - val_loss: 0.9028 - val_acc: 0.6747
Epoch 3/100
66228/66228 [==============================] - 35s 523us/step - loss: 0.8865 - acc: 0.6855 - val_loss: 0.9075 - val_acc: 0.6793
Epoch 4/100
66228/66228 [==============================] - 35s 527us/step - loss: 0.8665 - acc: 0.6899 - val_loss: 0.9015 - val_acc: 0.6805
Epoch 5/100
66228/66228 [==============================] - 35s 522us/step - loss: 0.8458 - acc: 0.6968 - val_loss: 0.8997 - val_acc: 0.6793
Epoch 6/100
66228/66228 [==============================] - 35s 522us/step - loss: 0.8276 - acc: 0.7017 - val_loss: 0.8917 - val_acc: 0.6803
Epoch 7/

Training and testing the second subset of data using Neural Networks.

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test = train_test_split(reviews['title'],train_size=0.8,random_state=1)
y_train,y_test = train_test_split(reviews['rating'],train_size=0.8,random_state=1)

In [0]:
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(x_train)
train_X = pad_sequences(tokenizer.texts_to_sequences(x_train), 33)
test_X = pad_sequences(tokenizer.texts_to_sequences(x_test),33)
train_Y=keras.utils.to_categorical(y_train)
test_Y=keras.utils.to_categorical(y_test)

In [0]:
embed_dim = 128
lstm_out = 196
max_features = 2000
model = Sequential()
model.add(Embedding(tokenizer.num_words, tokenizer.num_words, input_length=33))
model.add(Conv1D(66, kernel_size=3, activation='relu'))
model.add(Conv1D(66, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(.5))
model.add(Conv1D(33, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(.5))
model.add(Dense(6,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [0]:
batch_size = 32
model.fit( train_X,#.reshape((train_X.shape[0], train_X.shape[1],1)), 
           train_Y,
           epochs = 100, 
           batch_size=batch_size, 
           verbose = 1, 
           validation_data=(test_X, test_Y))

Train on 66228 samples, validate on 16557 samples
Epoch 1/100
66228/66228 [==============================] - 37s 557us/step - loss: 0.9944 - acc: 0.6566 - val_loss: 0.8749 - val_acc: 0.6884
Epoch 2/100
66228/66228 [==============================] - 36s 544us/step - loss: 0.8824 - acc: 0.6946 - val_loss: 0.8592 - val_acc: 0.6989
Epoch 3/100
66228/66228 [==============================] - 36s 546us/step - loss: 0.8565 - acc: 0.7033 - val_loss: 0.8474 - val_acc: 0.7010
Epoch 4/100
66228/66228 [==============================] - 36s 548us/step - loss: 0.8387 - acc: 0.7093 - val_loss: 0.8445 - val_acc: 0.7010
Epoch 5/100
66228/66228 [==============================] - 39s 584us/step - loss: 0.8258 - acc: 0.7135 - val_loss: 0.8447 - val_acc: 0.7014
Epoch 6/100
66228/66228 [==============================] - 38s 571us/step - loss: 0.8147 - acc: 0.7169 - val_loss: 0.8491 - val_acc: 0.7016
Epoch 7/100
66228/66228 [==============================] - 36s 538us/step - loss: 0.8042 - acc: 0.7206 - val_l